# <h1 style="text-align:center">Body Fat Calculator </h1>
<h4 style="text-align:right">Jiawen Chen, Chunyuan Jin, Han Liao </h4>



 
# 1 Introduction


## 1.1 Background

<p> Body fat percentage is a measure of fitness level, which is not easy to obtain during clinical application. A popular way to estimate the body fat is by using the Siri's equation. However, body density is also hard to obtain. 
    
<p> This project aims to come up with a simple, robust, accurate and precise “rule-of-thumb” method to estimate percentage of body fat using clinically available measurements.

## 1.2 Description of Dataset


### 1.2.1 Formula

<p> The Siri's equation for estimating the **body fat B(%)** is

 $$ Body\ Fat\ \%\ (i.e. 100 \times B) = \frac{495}{D} - 450,\ D = Body\ Density\ (gm/cm^{3})$$ 
 

<p> The formula for estimating **ADIPOSITY (BMI)** is
 
$$ADIPOSITY\ (BMI)= \frac{Weight\ (lbs) \times 703}{[Height\ (inch)]^2}$$

### 1.2.2 Overview of the dataset

We have **252** observations and **17** variables, their units are given in the parenthesis.
 
* Response variable: Body fat in percentage
* Predictors: Age (years), Weight (lbs), Height (inches), Adiposity (bmi), Neck circumference (cm), Chest circumference (cm), Abdomen circumference (cm), Hip circumference (cm), Thigh circumference (cm), Knee circumference (cm), Ankle circumference (cm), Biceps (extended) circumference (cm), Forearm circumference (cm), Wrist circumference (cm)

# 2 Data Cleaning

Our data cleaning process follows these criteria:

1. Impute those variables which have abnormal values and can be fixed with their redundant variable.
2. Filter out records which are abnormal and cannot be fixed.

## 2.1 Use Summary Table to Check Extreme Values

We use the summary table to get an overview of the data. What surprises us is the extreme values in some variables

From the summary table, we can see there exist extreme values in the some variables.

<img src="plots/summary.png" alt="Drawing" style="width: 500px;"/>

The observations that have extreme values are 

<img src="plots/data1.png" alt="Drawing" style="width: 750px;"/>

**BODYFAT** is the response variable whose reasonable value ranges from 2% to 39%. 
Individual 172 has lowest possible body fat, which can be considered as essential fat;
Individual 216 is sever obesity, which is possible;
Individual 182, it's impossible to have 0% of bodyfat, and after checking the siri's equation to his density, the corresponding bodyfat becomes negative, thus we filter this records out of our analysis.

There also exists extreme value in **WEIGHT**, which occurs in individual 39. This man also has the largest value in  ADIPOSITY, NECK, CHEST, ABDOMEN, HIP, THIGH, KNEE, BICEPS, and WRIST. Which indicates that this record does exist.

As for **HEIGHT**, individual 42's height is only 29.5 which is quite abnormal. After checking the corresponding weight by BMI formula, we can assume that this is a wrong record. Thus we fix his height by applying the BMI formula.


## 2.2 Check Siri's Equation

Known that the body fat percentage can be estimated by the density with the Siri's equation. We build a linear model between the bodyfat percentage estimated by Siri's equation and the bodyfat percentage in the data set. The residual plot and the QQ plot of this model is shown below. We can see that record 48, 76, 96 are possible outliers. 

<img src="plots/data2_1.png" alt="Drawing" style="width: 750px;"/>

Individual 96's other variables all have normal value, which indicates his desity might be wrongly recorded.

Individual 48 and 76 have similar values in other variables, thus their body fat percentage should also be similar. Thus we use the Siri's equation to fix their body fat percentage.

## 2.3 Check the BMI Formula

Known that the ADIPOSITY can be estimated using WEIGHT and HEIGHT. We build a linear model between the BMI estimated by equation and the ADIPOSITY in the data set. The residual plot and the QQ plot of this model is shown below. We can see that record 163, 220, 234 are possible outliers.

<img src="plots/data3_1.png" alt="Drawing" style="width: 750px;"/>

For individual 163, 220, and 234, their weights vary a lot, which means their BMI should not be similar to each other. Thus we adopt the calculated BMI as their ADIPOSITY.

## 2.3 Data cleaning summary
 
In conclusion:
    
Record **182** is filtered out because it has 0 body fat and there's no way to fix that.
    
The HEIGHT of record **42** is fixed according to the weight and adiposity.
    
The body fat of record **48** and **76** are fixed according to the density.
    
The adiposity of record **163**, **220**, and **234** are fixed according to the weight and height.

# 3 Variable Selection and Statistical Modeling

## 3.1 Variable Selection 
### Stepwise Backward and Forward LR
Considering there is a tradeoff between model variance and accuracy, we'd better not use all 14 predictors to establish the model. Although the mean square of error (MSE) will be small, the model itself will be unstable. Firstly, we try normal linear regression using the stepwise method to select features. We chose BIC as criterions and directions of forward and backward. The result was shown respectively.

In [14]:
BodyFatData <-read.csv("data/CleanData.csv")
summary(lm(BODYFAT ~ WEIGHT + ABDOMEN + FOREARM + WRIST, data = BodyFatData[, c(2, 4:17)]))#  use backward stepwise method with BIC

X1,X12.6,X1.0708,X23,X154.25,X67.75,X23.7,X36.2,X93.1,X85.2,X94.5,X59,X37.3,X21.9,X32,X27.4,X17.1
<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,6.9,1.0853,22,173.25,72.25,23.4,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
3,24.6,1.0414,22,154.00,66.25,24.7,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
4,10.9,1.0751,26,184.75,72.25,24.9,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
5,27.8,1.0340,24,184.25,71.25,25.6,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7
6,20.6,1.0502,24,210.25,74.75,26.5,39.0,104.5,94.4,107.8,66.0,42.0,25.6,35.7,30.6,18.8
7,19.0,1.0549,26,181.00,69.75,26.2,36.4,105.1,90.7,100.3,58.4,38.3,22.9,31.9,27.8,17.7


The results of backward and forward method are the same, WEIGHT, ABDOMEN, FOREARM, WRIST were chosen as predictors and they are all significant. R-square of the model is 0.736, which indicates the accuracy of the model is quite good.  However,  in the linear model, colinearity of predictors will increase the variance of model. VIF is a value to identify colinearity. There are colineary between variables if VIF is larger than 5.

In [16]:
library(car)
LR_backward<-lm(formula = BODYFAT ~ ABDOMEN + WEIGHT + WRIST + FOREARM, 
            data = BodyFatData[, c(2,4:17)])
vif(LR_backward)>5

ABDOMEN  WEIGHT   WRIST FOREARM 
  FALSE    TRUE   FALSE   FALSE

The model is still unstable, so we try other methods to do variable selection.

### Variable Selection with XGBoost
XGBoost is an ensemble learning method with tree models. It will output the importance of variables after establishing the model. The criterion for ranking predictors is how many times the variable is chosen as the node in decision trees. The more frequent it is chosen the larger score it will get. Moreover, if the variable is chosen near root of tree, the score will has a large weight because its influence to the decision tree is large.

![](plots/xgboost.png)

## 3.2 Stable Statistical Model
In both of our methods, we find that ABDOMEN is the most important variable among 14 predictors. WEIGHT ranks high in both methods comparing with other variables. WRIST is significant in linear stepwise model and ADIPOSITY scores high in nonlinear model. Considering the rule of thumb, we select ABDOMEN as our first variable in predicting body fat. Also, to increase the accuracy we choose another variable among WRIST, WEIGHT, ADIPOSITY as our second variable. The number of predictors are too small to use any tree model or ensemble method in machine learning. We still choose normal linear regression as our model. The colinearity are not significant between two variables, so we may not add regulization to our model. After comparing three models, we choose ABDOMEN and WEIGHT as our predictors, because it has highest R-square 0.72.

In [21]:
fit1<-lm(formula = BODYFAT ~ ABDOMEN + WEIGHT, 
            data = BodyFatData[, c("BODYFAT", "WEIGHT", "ABDOMEN")])
summary(fit1)


Call:
lm(formula = BODYFAT ~ ABDOMEN + WEIGHT, data = BodyFatData[, 
    c("BODYFAT", "WEIGHT", "ABDOMEN")])

Residuals:
     Min       1Q   Median       3Q      Max 
-10.3183  -2.9808   0.0981   2.8756   9.6169 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -40.77528    2.39583 -17.019  < 2e-16 ***
ABDOMEN       0.91797    0.05203  17.642  < 2e-16 ***
WEIGHT       -0.14060    0.01911  -7.358 2.78e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.063 on 246 degrees of freedom
Multiple R-squared:  0.7206,	Adjusted R-squared:  0.7183 
F-statistic: 317.2 on 2 and 246 DF,  p-value: < 2.2e-16


## 3.3 Model Diagnostics
Firstly, use VIF to check colinearity. There is no obvious colinearity between ABDOMEN and WEIGHT.

In [22]:
vif(fit1)>5

ABDOMEN  WEIGHT 
  FALSE   FALSE

Then, plot the residuals vs fitted values. Residuals are seperated randomly near 0 and there is no correlation between fitted value and residuals. Also, there is no correlation between residuals and predictors.
![](plots/res.png)

QQ-plot of residuals show that residuals generally follows normal distribution. It is normal that there are still some outliers in the model.
![](plots/outlier.png)

# 4 Conclusion


 
## 4.1 Rule of Thumb 

Multiply your abdomen (cm) by.

$$ Body\ Fat\ \% =  -40.77528 + 0.91797*ABDOMEN (cm) - 0.14060*WEIGHT(lbs) $$



## 4.2 Strength & Weekness

Our model has the following strength:

1. **Simplicity**

2. **Robustness**

3. **Interpretability**

As for the weekness:

**Only use two predictors, some information from other predictors may be ignored.**





# 5 Contribution

Jiawen Chen: 

Chunyuan Jin:

Han Liao: